In [18]:
PATH_PROJECT = ""
DATASET_PATH = PATH_PROJECT + "dataset/"
RSS_FILENAME = DATASET_PATH + "rss_dataset.csv"
SIMPLE_FEATURES_FILENAME = DATASET_PATH + "simple_f_dataset.csv"

# LENTA_ARCHIVE_RSS = DATASET_PATH + "src_rss_archive/lenta-ru-news.csv"
LENTA_ARCHIVE_RSS = DATASET_PATH + "src_rss_archive/clear_mini_lenta_ru_news_2.csv"


In [11]:

import pandas as pd
import re
import numpy as np

import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

nltk.download('stopwords')
rus_stopwords = stopwords.words("russian")
snowball = SnowballStemmer(language="russian")
nltk.download('punkt')

import pymorphy2

morph = pymorphy2.MorphAnalyzer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
# news_df = pd.read_csv(RSS_FILENAME, sep=";", index_col=["id"])
# news_df

In [23]:
class TextProcessing():
    """
    Делаем следующую обрабатку текста:
    - downcase
    - лемматизация - боримся с окончаниями
    - убираем стоп-слова
    - стемминг - обрезаем окончания (опционально)
    """

    def __init__(self ):
        pass
    
    def preprocessing_text(self, text):
        # приводим к нижнему регистру
        text = text.lower()
        # убираем стоп слова
        text = " ".join([w for w in re.findall(r"\w+", text) if w not in rus_stopwords])
        # лемматизация
        text = " ".join([morph.parse(w)[0].normal_form for w in text.split(' ')])
        # стемминг
        text = " ".join([snowball.stem(w) for w in text.split(' ')])
        return text
    
    def preprocessing_df(self, df, columns=["description", "title", "summary"], enable_stemming=False):
        self.df = df
        # Заполняем пустые значениям описания заголовком статьи
        self.fillna_columns()
        for column_name in columns:
            self.preprocessing_series(column=column_name, enable_stemming=enable_stemming)
        return self.df
    
    def fillna_columns(self):
        df = self.df
        
        # Для случаев когда summary пустой заполняем description
        df["summary"] = df["summary"].fillna(df["description"])
        # Для случаев когда summary пустой заполняем title
        # Для случаев когда и description пустой использеум title
        df["summary"] = df["summary"].fillna(df["title"])
        df["description"] = df["description"].fillna(df["summary"])
        df["title"] = df["title"].fillna(df["summary"])
    
    def preprocessing_series(self, column, enable_stemming=False):
        # приводим к нижнему регистру
        self.df[f"{column}_preproc"] = self.lower_text(self.df[column])
        # убираем стоп слова
        self.df[f"{column}_preproc"] = self.stopwords(self.df[f"{column}_preproc"])
        # лемматизация
        self.df[f"{column}_preproc"] = self.lemmatization(self.df[f"{column}_preproc"])
        # стемминг
        if enable_stemming:
            self.df[f"{column}_preproc"] = self.stemming(self.df[f"{column}_preproc"])

    def lower_text(self, series):
        # приводим к нижнему регистру
        return series.str.lower()

    def stopwords(self, series):
        lemma_series = series.apply(lambda x: " ".join([w for w in re.findall(r"\w+", x) if w not in rus_stopwords]))
        return lemma_series

    def lemmatization(self, series):
        # лемматизация
        lemma_series = series.apply(lambda x: " ".join([morph.parse(w)[0].normal_form for w in x.split(' ')]))
        return lemma_series

    def stemming(self, series):
        # стемминг
        stemm_series = series.apply(lambda x: " ".join([snowball.stem(w) for w in x.split(' ')]))
        return stemm_series


In [ ]:
booker = ["налог", "контролирующие органы", "МСФО", "юрлиц", "физлиц", "инспекция", "налоговик", "налогоплательщик", "налоговый учет", "баланс", "бюджетирование", "ИФНС", "ПФР", "ФСС", "ФОТ", ]
booker

In [84]:
LENTA_DATASET_PATH = DATASET_PATH + "src_rss_archive/"


In [95]:
lenta_ds_filename = "clear_mini_lenta_ru_news_5.csv"
lenta_df = pd.read_csv(LENTA_DATASET_PATH + lenta_ds_filename, sep=",", header=None, names=["link","title","description","summary","tags","published_date"])
lenta_df.head(3)


,link,title,description,summary,tags,published_date
0,https://lenta.ru/news/2017/07/21/povar/,TAG Heuer сделал шеф-повара посланником бренда...,Новым посланником бренда TAG Heuer в России ст...,Ценности,Явления,2017/07/21
1,https://lenta.ru/news/2017/07/21/tours_earthqu...,Россияне начали отказываться от туров в Грецию...,На горячую линию Ростуризма поступают многочис...,Путешествия,Происшествия,2017/07/21
2,https://lenta.ru/news/2017/07/21/nymuseum/,Louis Vuitton покажет в Париже полотна из нью-...,Французский модный дом Louis Vuitton и его под...,Ценности,Явления,2017/07/21


In [96]:

# ПредОбработка текста
txt_proc = TextProcessing()
lenta_df = txt_proc.preprocessing_df(lenta_df, columns=["description"])

lenta_df


,link,title,description,summary,tags,published_date,description_preproc
0,https://lenta.ru/news/2017/07/21/povar/,TAG Heuer сделал шеф-повара посланником бренда...,Новым посланником бренда TAG Heuer в России ст...,Ценности,Явления,2017/07/21,новый посланник бренд tag heuer россия стать б...
1,https://lenta.ru/news/2017/07/21/tours_earthqu...,Россияне начали отказываться от туров в Грецию...,На горячую линию Ростуризма поступают многочис...,Путешествия,Происшествия,2017/07/21,горячий линия ростуризм поступать многочисленн...
2,https://lenta.ru/news/2017/07/21/nymuseum/,Louis Vuitton покажет в Париже полотна из нью-...,Французский модный дом Louis Vuitton и его под...,Ценности,Явления,2017/07/21,французский модный дом louis vuitton подраздел...
3,https://lenta.ru/news/2017/07/21/shangaibuildi...,В Шанхае рухнуло предназначенное под снос здание,В одном из крупнейших городов Китая Шанхае рух...,Мир,Происшествия,2017/07/21,один крупный город китай шанхай рухнуть предна...
4,https://lenta.ru/news/2017/07/21/eremacas/,CAS оставил в силе дисквалификацию пойманного ...,Спортивный арбитражный суд (CAS) оставил в сил...,Спорт,Футбол,2017/07/21,спортивный арбитражный суд cas оставить сила р...
...,...,...,...,...,...,...,...
29274,https://lenta.ru/news/2018/02/21/ghouta/,Раскрыты факты об истреблении мирных жителей в...,Верховный комиссар ООН по правам человека Зейд...,Мир,Конфликты,2018/02/21,верховный комиссар оон право человек зейд раад...
29275,https://lenta.ru/news/2018/02/21/capitan/,Бравый капитан устроил коллапс в российском порту,Отгрузка нефти в Новороссийском морском торгов...,Экономика,Рынки,2018/02/21,отгрузка нефть новороссийский морской торговый...
29276,https://lenta.ru/news/2018/02/21/okr_shtraf/,Россия заплатила МОК 15 миллионов долларов,Олимпийский комитет России (ОКР) заплатил Межд...,Спорт,NaN,2018/02/21,олимпийский комитет россия окр заплатить между...
29277,https://lenta.ru/news/2018/02/21/csr/,Расходы на здравоохранение решили переложить н...,В России нужно увеличивать финансирование меди...,Экономика,Госэкономика,2018/02/21,россия нужно увеличивать финансирование медици...


In [103]:
# lenta_ds_filename = "clear_mini_lenta_ru_news_5.csv"
for lenta_ds_filename in ["clear_mini_lenta_ru_news.csv", "clear_mini_lenta_ru_news_2.csv"]:
    print(lenta_ds_filename)
    lenta_df = pd.read_csv(LENTA_DATASET_PATH + lenta_ds_filename, sep=",", header=None, names=["link","title","description","summary","tags","published_date"])
    lenta_df.head(3)

    # ПредОбработка текста
    txt_proc = TextProcessing()
    lenta_df = txt_proc.preprocessing_df(lenta_df, columns=["description"])

    lenta_df

    def is_include(text, role_keywords = []):
        role_keywords = ["налог", "контролирующие органы", "МСФО", "юрлиц", "физлиц", "инспекция", "налоговик", "налогоплательщик", "налоговый учет", "баланс", "бюджетирование", "ИФНС", "ПФР", "ФСС", "ФОТ", ]
        role_keywords = [x.lower() for x in role_keywords]
        return sum([text.count(x) for x in role_keywords]) > 2

    good_tags = ['Деловой климат', 'Госэкономика', 'Социальная сфера', 'Общество', 'Бизнес', 'Рынки', 'Политика', 'Деньги', 'Экономика', 'Регионы',]
    lenta_df["target"] = "unknow"
    mask = lenta_df['description_preproc'].apply(lambda x: is_include(x))
    lenta_df["target"] = np.where(mask & lenta_df["tags"].isin(good_tags) , "booker", "unknow")

    find_positive_target = lenta_df[lenta_df["target"] == "booker"]
    count_find_target = find_positive_target.shape

    find_negative_target = lenta_df[~lenta_df["tags"].isin(good_tags)].sample(n=int(count_find_target[0]*1.5))

    union_find_target = pd.concat([find_positive_target, find_negative_target], ignore_index=True)
    union_find_target.to_csv(LENTA_DATASET_PATH + "target_" +  lenta_ds_filename, sep=";", encoding="utf-8-sig", header=True, index=True, index_label="id")


clear_mini_lenta_ru_news.csv
clear_mini_lenta_ru_news_2.csv


In [102]:
union_find_target

,link,title,description,summary,tags,published_date,description_preproc,target
0,https://lenta.ru/news/2016/02/17/scotland/,Шотландский банк ушел от налогов благодаря фил...,Королевский банк Шотландии (Royal Bank of Scot...,Экономика,Бизнес,2016/02/17,королевский банк шотландия royal bank of scotl...,booker
1,https://lenta.ru/news/2016/02/17/vodka/,СМИ сообщили о начале зачистки алкогольного рынка,"Сотрудники ФСБ и ФНС во вторник, 16 февраля, п...",Бизнес,Бизнес,2016/02/17,сотрудник фсб фнс вторник 16 февраль провести ...,booker
2,https://lenta.ru/news/2016/02/25/nalogi/,Шувалов допустил изменение налоговой системы,Конфигурация налоговой системы может быть изме...,Экономика,Госэкономика,2016/02/25,конфигурация налоговый система изменить оконча...,booker
3,https://lenta.ru/news/2016/02/25/freelancepens/,Фрилансеров и фермеров предложили вывести из п...,Государственная система обязательного пенсионн...,Экономика,Деньги,2016/02/25,государственный система обязательный пенсионны...,booker
4,https://lenta.ru/news/2016/03/01/avto_nalog/,Путин утвердил повышение акцизов на автомобили,Президент России Владимир Путин утвердил попра...,Бизнес,Деловой климат,2016/03/01,президент россия владимир путин утвердить попр...,booker
...,...,...,...,...,...,...,...,...
627,https://lenta.ru/news/2016/04/28/tysonfury/,Фьюри покинет Великобританию из-за травли цыган,"Британский боксер Тайсон Фьюри заявил, что пос...",Спорт,Бокс и ММА,2016/04/28,британский боксёр тайсон фьюри заявить бой рев...,unknow
628,https://lenta.ru/news/2016/02/25/waves/,Гравитационные волны Вселенной обнаружат с пом...,"Астрофизики из США и Германии показали, что пр...",Наука и техника,Космос,2016/02/25,астрофизик сша германия показать помощь радиот...,unknow
629,https://lenta.ru/news/2016/08/14/planets/,В окольцевании кентавров обвинили гигантов,"Астрономы из Франции и Японии показали, что ко...",Наука и техника,Космос,2016/08/14,астроном франция япония показать кольцо вокруг...,unknow
630,https://lenta.ru/news/2016/08/29/parispolicesh...,Полицейские открыли огонь по подозрительной ма...,Парижские полицейские открыли огонь по подозри...,Мир,Преступность,2016/08/29,парижский полицейский открыть огонь подозрител...,unknow


In [15]:
# gen_train_data.ipynb - обновлен класс по предобработке текста: дозаполнение пустых данных, перевод в нижний регистр, стоп слова, лемматизация, стемминг (опционально). Пока в виде юпитерноутбука, поозже в отдельный модуль будет перенесено

In [7]:
    

# ПредОбработка текста
txt_proc = TextProcessing()
news_df = txt_proc.preprocessing_df(news_df)

news_df


,news_for_role,title,tags,summary,description,link,published_dt,published_ts,description_preproc,title_preproc,summary_preproc
id,,,,,,,,,,,
0,booker,Ошибки в данных физлиц - недостоверные сведени...,NaN,Организация представила в инспекцию справки 2-...,Организация представила в инспекцию справки 2-...,http://www.consultant.ru/law/review/208422377....,2022-10-07 12:00:00+03:00,1.665133e+09,организация представить инспекция справка 2 нд...,ошибка данные физлицо недостоверный сведение к...,организация представить инспекция справка 2 нд...
1,booker,Срок на возврат налоговой переплаты считают с ...,NaN,Организация обратилась к налоговикам за возвра...,Организация обратилась к налоговикам за возвра...,http://www.consultant.ru/law/review/208422378....,2022-10-07 12:00:00+03:00,1.665133e+09,организация обратиться налоговик возврат ндс к...,срок возврат налоговый переплата считать дата ...,организация обратиться налоговик возврат ндс к...
2,booker,В Москве приняли допмеры поддержки семей мобил...,NaN,Мэр Москвы принял ряд мер соцподдержки семей г...,Мэр Москвы принял ряд мер соцподдержки семей г...,http://www.consultant.ru/law/review/208422379....,2022-10-07 12:00:00+03:00,1.665133e+09,мэр москва принять ряд мера соцподдержка семья...,москва принять допмера поддержка семья мобилиз...,мэр москва принять ряд мера соцподдержка семья...
3,booker,"Суд поддержал работодателя, который согласовал...",NaN,Работодатель не обязан переносить отпуск сотру...,Работодатель не обязан переносить отпуск сотру...,http://www.consultant.ru/law/review/208422380....,2022-10-07 12:00:00+03:00,1.665133e+09,работодатель обязать переносить отпуск сотрудн...,суд поддержать работодатель который согласоват...,работодатель обязать переносить отпуск сотрудн...
4,booker,"Проект Приказа Минцифры России ""Об утверждении...",NaN,"Проект Приказа Минцифры России ""Об утверждении...","Проект Приказа Минцифры России ""Об утверждении...",http://www.consultant.ru/law/hotdocs/link/?id=...,2022-10-07 17:47:36+03:00,1.665154e+09,проект приказ минцифра россия утверждение поря...,проект приказ минцифра россия утверждение поря...,проект приказ минцифра россия утверждение поря...
...,...,...,...,...,...,...,...,...,...,...,...
108,owner,Константин Чуйченко встретился с Чрезвычайным ...,NaN,Константин Чуйченко встретился с Чрезвычайным ...,Константин Чуйченко встретился с Чрезвычайным ...,https://minjust.gov.ru/ru/events/49123/,2022-10-08 05:46:07+00:00,1.665208e+09,константин чуйченко встретиться чрезвычайный п...,константин чуйченко встретиться чрезвычайный п...,константин чуйченко встретиться чрезвычайный п...
109,owner,"Разработаны законопроекты, направленные на гум...",NaN,"Разработаны законопроекты, направленные на гум...","Разработаны законопроекты, направленные на гум...",https://minjust.gov.ru/ru/events/49122/,2022-10-08 05:46:07+00:00,1.665208e+09,разработать законопроект направить гуманизация...,разработать законопроект направить гуманизация...,разработать законопроект направить гуманизация...
110,owner,"В Республике Мордовия, Новгородской, Новосибир...",NaN,"В Республике Мордовия, Новгородской, Новосибир...","В Республике Мордовия, Новгородской, Новосибир...",https://minjust.gov.ru/ru/events/49121/,2022-10-08 05:46:07+00:00,1.665208e+09,республика мордовия новгородский новосибирский...,республика мордовия новгородский новосибирский...,республика мордовия новгородский новосибирский...


In [16]:
booker = ["налог", "контролирующие органы", "МСФО", "юрлиц", "физлиц", "инспекция", "налоговик", "налогоплательщик", "налоговый учет", "баланс", "бюджетирование", "ИФНС", "ПФР", "ФСС", "ФОТ", ]
booker

['налог',
 'контролирующие органы',
 'МСФО',
 'юрлиц',
 'физлиц',
 'инспекция',
 'налоговик',
 'налогоплательщик',
 'налоговый учет',
 'баланс',
 'бюджетирование',
 'ИФНС',
 'ПФР',
 'ФСС',
 'ФОТ']

In [24]:
# ПредОбработка текста
txt_proc = TextProcessing()
lenta_df = txt_proc.preprocessing_df(lenta_df, columns=["summary"])

lenta_df


,link,title,description,summary,tags,published_date,description_preproc,summary_preproc
0,https://lenta.ru/news/2018/10/27/medvedeva_skate/,Медведева показала худший результат в карьере,Российская фигуристка Евгения Медведева заняла...,Спорт,Зимние виды,2018/10/27,российская фигуристка евгения медведева заняла...,спорт
1,https://lenta.ru/news/2018/10/27/sdays/,Ассанж назвал условия капитуляции перед британ...,Основатель WikiLeaks Джулиан Ассанж готов сдат...,Мир,Политика,2018/10/27,основатель wikileaks джулиан ассанж готов сдат...,мир
2,https://lenta.ru/news/2018/10/27/sposob/,Таможенники разоблачили популярный метод ввоза...,Федеральная таможенная служба (ФТС) России объ...,Экономика,Все,2018/10/27,федеральная таможенная служба фтс россии объяс...,экономика
3,https://lenta.ru/news/2018/10/27/kokorin/,Брат Кокорина объяснил нападение на водителя в...,Младший брат футболиста Александра Кокорина Ки...,Спорт,Футбол,2018/10/27,младший брат футболиста александра кокорина ки...,спорт
4,https://lenta.ru/news/2018/10/27/miss/,Победительница конкурса красоты рухнула в обмо...,Финалистка конкурса Miss Grand International п...,Из жизни,Люди,2018/10/27,финалистка конкурса miss grand international п...,жизнь
...,...,...,...,...,...,...,...,...
22562,https://lenta.ru/news/2019/04/20/kokomama/,Появилось видео за минуту до избиения чиновник...,Мать нападающего петербургского «Зенита» Алекс...,Появилось видео за минуту до избиения чиновник...,Футбол,2019/04/20,мать нападающего петербургского зенита алексан...,появиться видео минута избиение чиновник кокор...
22563,https://lenta.ru/news/2019/04/20/oil/,Белоруссия потребует от России компенсацию за ...,Белоруссия обсудит с российской госкомпанией «...,Белоруссия потребует от России компенсацию за ...,Госэкономика,2019/04/20,белоруссия обсудит российской госкомпанией тра...,белоруссия потребовать россия компенсация плох...
22564,https://lenta.ru/news/2019/04/20/pastor/,Избит один из самых богатых пасторов России,Житель Санкт-Петербурга был избит и ограблен в...,Избит один из самых богатых пасторов России,Криминал,2019/04/20,житель санкт петербурга избит ограблен собстве...,избитый самый богатый пастор россия
22565,https://lenta.ru/news/2019/04/20/treaty/,Белоруссия раскрыла свой план по договору с Ро...,Белоруссия предлагает России не вносить концеп...,Белоруссия раскрыла свой план по договору с Ро...,Белоруссия,2019/04/20,белоруссия предлагает россии вносить концептуа...,белоруссия раскрыть свой план договор россия


In [25]:
lenta_df.shape

(22567, 8)

In [64]:
import numpy as np
def is_include(text, role_keywords = []):
    role_keywords = ["налог", "контролирующие органы", "МСФО", "юрлиц", "физлиц", "инспекция", "налоговик", "налогоплательщик", "налоговый учет", "баланс", "бюджетирование", "ИФНС", "ПФР", "ФСС", "ФОТ", ]
    role_keywords = [x.lower() for x in role_keywords]
#     role_keywords = ["сбербанк"]
#     return any([x in text for x in role_keywords])
#     print( [text.count(x) for x in role_keywords] )
#     assert False
    
    return sum([text.count(x) for x in role_keywords]) > 2
#     return sum([x in text for x in role_keywords]) > 2


booker = ["сбербанк"]
# np.where(lenta_df.loc[0:5]["description_preproc"])
# is_include(lenta_df.loc[0:5]["description_preproc"])
# np.where(is_include(lenta_df.loc[0:5]["description_preproc"], booker))
# lenta_df[is_include(lenta_df.loc[0:5]["description_preproc"], booker)]
# booker
good_tags = ['Деловой климат', 'Госэкономика', 'Социальная сфера', 'Общество', 'Бизнес', 'Рынки', 'Политика', 'Деньги', 'Экономика', 'Регионы',]
lenta_df["target"] = "unknow"
mask = lenta_df['description_preproc'].apply(lambda x: is_include(x))
# lenta_df.loc[lenta_df["description_preproc"].apply(lambda x: is_include(x))]

# lenta_df[lenta_df[mask]]["target"] = "booker"
lenta_df["target"] = np.where(mask & lenta_df["tags"].isin(good_tags) , "booker", "unknow")
# lenta_df["target"] = np.where((lenta_df["target"] == "booker")  & lenta_df["tags"].isin(good_tags), "booker", "unknow")
lenta_df[mask]["target"]
# np.where(lenta_df[mask]["description_preproc"].isin(good_tags), "booker", "unknow")

# print(mask)
# lenta_df[mask]["tags"] in ['Следствие и суд'])
# lenta_df["target"] = "none"
# lenta_df[mask][lenta_df["tags"].isin(good_tags)]["target"] = "booker"

# lenta_df[mask][lenta_df["tags"].isin(good_tags)]["target"]

# print(lenta_df[mask].shape)
# lenta_df.loc[mask].shape


# lenta_df[mask]["tags"].unique()
# list(lenta_df[mask]["description_preproc"])



52       booker
65       unknow
197      booker
228      booker
236      unknow
          ...  
22337    unknow
22415    unknow
22419    unknow
22473    unknow
22527    unknow
Name: target, Length: 429, dtype: object

In [67]:
lenta_df[lenta_df["target"] == "booker"]
# ["tags"].unique()
# lenta_df[mask]["tags"].unique()
# lenta_df.shape
lenta_df.to_csv
df.to_csv(clear_mini_lenta_ru_news_2.csv, sep=";", encoding="utf-8-sig", header=True, index=True, index_label="id")



,link,title,description,summary,tags,published_date,description_preproc,summary_preproc,target
52,https://lenta.ru/news/2018/10/28/kredito/,Миллион россиян не сможет выехать из страны из...,Более миллиона жителей России не смогут выехат...,Россия,Общество,2018/10/28,миллиона жителей россии смогут выехать пределы...,россия,booker
197,https://lenta.ru/news/2018/10/29/eshe_nalog/,Богатых россиян заставят раскошелиться,Министерство финансов России будет повышать на...,Экономика,Госэкономика,2018/10/29,министерство финансов россии повышать налоги р...,экономика,booker
228,https://lenta.ru/news/2018/10/29/neftalzis/,Эксперт оценил последствия повышения акцизов н...,"Увеличение ставок сразу трех налогов, которое ...",Экономика,Деловой климат,2018/10/29,увеличение ставок сразу трех налогов которое п...,экономика,booker
297,https://lenta.ru/news/2018/10/30/keep_calm/,Богатых россиян успокоили,Министерство финансов на самом деле не планиру...,Экономика,Госэкономика,2018/10/30,министерство финансов самом деле планирует пер...,экономика,booker
499,https://lenta.ru/news/2018/10/31/tankaev/,Рост цен на бензин приведет к снижению качеств...,Рост цен на бензин в стране приведет к понижен...,Экономика,Деловой климат,2018/10/31,рост цен бензин стране приведет понижению потр...,экономика,booker
...,...,...,...,...,...,...,...,...,...
21796,https://lenta.ru/news/2019/04/16/rosneft11/,Совет директоров «Роснефти» дал рекомендации Г...,Совет директоров «Роснефти» утвердил годовой о...,Совет директоров «Роснефти» дал рекомендации Г...,Деловой климат,2019/04/16,совет директоров роснефти утвердил годовой отч...,совет директор роснефть дать рекомендация гос ...,booker
21982,https://lenta.ru/news/2019/04/17/dm_biz/,На нацпроекты привлекли 75 триллионов рублей,"Премьер-министр Дмитрий Медведев заявил, что к...",На нацпроекты привлекли 75 триллионов рублей,Госэкономика,2019/04/17,премьер министр дмитрий медведев заявил крупны...,нацпроект привлечь 75 триллион рубль,booker
22004,https://lenta.ru/news/2019/04/17/flower/,Родившую в первые минуты года россиянку обязал...,"Жительница Омска, родившая одного из первых в ...",Родившую в первые минуты года россиянку обязал...,Общество,2019/04/17,жительница омска родившая одного первых 2019 г...,родить первый минута год россиянка обязать зап...,booker
22063,https://lenta.ru/news/2019/04/17/nalogg/,Чиновники оплатили налог за цветы родившей в п...,Российские чиновники сами оплатили налог за цв...,Чиновники оплатили налог за цветы родившей в п...,Общество,2019/04/17,российские чиновники сами оплатили налог цветы...,чиновник оплатить налог цветок родить первый м...,booker


In [ ]:


lenta_df.loc[0]["description_preproc"]

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
lenta_df["tokens_description"] = lenta_df["description_preproc"].apply(nltk.word_tokenize)
lenta_df.head(4)

In [ ]:
text = lenta_df.loc[0]["description_preproc"]
tokens = word_tokenize(lenta_df.loc[0]["description_preproc"])
tokens

In [ ]:
tokens = lenta_df.loc[0]["tokens_description"]
unigram = list(nltk.ngrams(tokens, 1))
bigram = list(nltk.ngrams(tokens, 2))
trigram = list(nltk.ngrams(tokens, 3))
print(unigram[:5])
print(bigram[:5])
print(trigram[:5])


In [ ]:
txt_proc = TextProcessing()
booker


In [ ]:
#  with open(LENTA_ARCHIVE_RSS, mode="r", encoding="utf_8_sig") as f:
#         print(f.readline())
#         my_lines = f.readline()
#         print(my_line)
#         my_lines = f.readline()
#         print(my_line)
#         print(my_lines.split(","))
#         url,title,text,topic,tags,date = my_lines.split(",")
#         print(title)
#         print(text)

#         print("---")
#         print(txt_proc.preprocessing_text(title))
#         print(txt_proc.preprocessing_text(text))
#         print("---")
        
        
# #         print(f.readline())
# #         print(f.readline())
#         sys.exit()
# 1+2